## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
MODULE_BASE_PATH = '/kaggle/input/bert-base-uncased-huggingface/bert-base-uncased'
MODULE_PATH = MODULE_BASE_PATH + '/bert-base-uncased-tf_model.h5'
VOCAB_PATH = MODULE_BASE_PATH + '/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*last_epoch.h5')
model_path_list += glob.glob('/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_1_last_epoch.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_2_last_epoch.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_3_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_1_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_2_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

In [4]:
### BERT auxiliar function
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length, ignore_first_sep=True):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if ignore_first_sep:
                ignore_first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input_2_tail(ft_input1, ft_input2, tokenizer, max_sequence_length, 
                  max_len_ft1=254, max_len_ft2=255):

    ft1 = tokenizer.tokenize(ft_input1)
    ft2 = tokenizer.tokenize(ft_input2)
    
    ft1_len = len(ft1)
    ft2_len = len(ft2)

    if (ft1_len + ft2_len + 3) > max_sequence_length:
      
        if max_len_ft2 > ft2_len:
            ft2_new_len = ft2_len 
            ft1_new_len = max_len_ft1 + (max_len_ft2 - ft2_len)
        elif max_len_ft1 > ft1_len:
            ft2_new_len = max_len_ft2 + (max_len_ft1 - ft1_len)
            ft1_new_len = ft1_len
        else:
            ft2_new_len = max_len_ft2
            ft1_new_len = max_len_ft1
                        
        if ft1_new_len + ft2_new_len + 3 > max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (ft1_new_len + ft2_new_len + 3)))
        
        ft1 = ft1[-ft1_new_len:]
        ft2 = ft2[-ft2_new_len:]
        
    return ft1, ft2

def _convert_to_bert_inputs_2(feature1, feature2, tokenizer, max_sequence_length, ignore_first_sep=True):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + feature1 + ["[SEP]"] + feature2 + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length, ignore_first_sep)

    return [input_ids, input_masks, input_segments]

def compute_input_arays_2_tail(df, columns, tokenizer, max_sequence_length, 
                          max_len_ft1, max_len_ft2, ignore_first_sep=True):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in df[columns].iterrows():
        ft1, ft2 = instance[columns[0]], instance[columns[1]]

        ft1, ft2 = _trim_input_2_tail(ft1, ft2, tokenizer, max_sequence_length, 
                                      max_len_ft1, max_len_ft2)

        ids, masks, segments = _convert_to_bert_inputs_2(ft1, ft2, tokenizer, max_sequence_length, 
                                                         ignore_first_sep)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

## Pre-process data

In [5]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# Model parameters

In [6]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [7]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features HEAD
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Test features TAIL
X_test_tail = compute_input_arays_2_tail(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                                         MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
              + compute_input_arays_2_tail(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                           MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [8]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_model = TFBertModel.from_pretrained(MODULE_PATH, config=bert_config)
    sequence_output_question, pooled_output_question = bert_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = bert_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [9]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += ((model.predict(X_test) + model.predict(X_test_tail)) / 2) / len(model_path_list)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test


n=test['url'].apply(lambda x:(('ell.stackexchange.com' in x) or ('english.stackexchange.com' in x))).tolist()
spelling=[]
for x in n:
    if x:
        spelling.append(0.5)
    else:
        spelling.append(0.)
        
submission['question_type_spelling']=spelling


submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.951130,0.584729,0.225653,0.560831,0.477270,0.531714,0.725665,0.683357,0.598094,...,0.912980,0.927229,0.573676,0.969480,0.974362,0.807565,0.014231,0.010734,0.855586,0.921354
1,46,0.887692,0.513946,0.005960,0.781413,0.759154,0.932454,0.559110,0.507134,0.080268,...,0.656882,0.938072,0.641779,0.973540,0.981686,0.886974,0.932608,0.132009,0.077618,0.900711
2,70,0.921599,0.711387,0.027125,0.760555,0.905754,0.954795,0.635564,0.503761,0.135382,...,0.897837,0.946749,0.616952,0.972919,0.977549,0.866551,0.016880,0.023228,0.882225,0.899384
3,132,0.877739,0.429567,0.006347,0.720044,0.724184,0.923836,0.535184,0.408150,0.079813,...,0.711565,0.949156,0.706888,0.964335,0.975601,0.916467,0.747019,0.128595,0.684756,0.910878
4,200,0.939373,0.465913,0.027192,0.906229,0.777393,0.939523,0.617674,0.638749,0.110038,...,0.617279,0.911116,0.646950,0.968119,0.963684,0.802524,0.292091,0.131279,0.539301,0.918947


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.904865,0.604279,0.038994,0.748374,0.803292,0.860240,0.589755,0.502048,0.227051,...,0.805808,0.940462,0.668274,0.968781,0.973403,0.872921,0.513089,0.128422,0.524782,0.914153
std,2812.670060,0.043264,0.155141,0.074594,0.105487,0.115212,0.129591,0.058055,0.097772,0.224069,...,0.094174,0.030605,0.052948,0.014130,0.016712,0.051625,0.331113,0.062936,0.299793,0.025503
min,39.000000,0.761815,0.265289,0.002356,0.220559,0.340819,0.180058,0.489821,0.321297,0.006368,...,0.538180,0.758484,0.499184,0.880033,0.816340,0.623910,0.003777,0.007956,0.020136,0.760826
25%,2572.000000,0.876849,0.476016,0.007053,0.694116,0.749710,0.840742,0.545087,0.429261,0.056864,...,0.730413,0.927522,0.634224,0.963123,0.968251,0.849292,0.161892,0.081869,0.262515,0.901957
50%,5093.000000,0.906996,0.581447,0.011254,0.753893,0.821374,0.906667,0.577647,0.472852,0.135905,...,0.804221,0.947982,0.666274,0.971787,0.977964,0.884824,0.594713,0.132395,0.536580,0.919054
75%,7482.000000,0.938803,0.734959,0.027680,0.812678,0.879189,0.934945,0.622500,0.554410,0.346932,...,0.895527,0.961177,0.698810,0.978126,0.983408,0.910462,0.821950,0.171049,0.802792,0.931649
max,9640.000000,0.985607,0.929717,0.570365,0.979296,0.986861,0.984594,0.748017,0.798630,0.953597,...,0.966972,0.984792,0.837609,0.989516,0.994447,0.962881,0.964503,0.321225,0.989479,0.961655
